In [21]:
import random
import math
import numpy as np
from itertools import groupby as g
import operator


In [22]:
def price(A_t, B, q):
    return (A_t - q) / B

In [23]:
def competitive_quantity(n, A_t, alpha, beta, B, c): 
    return min(n*(A_t - alpha*B) / (n + beta*B), c*n)

In [24]:
def production_costs(q, alpha, beta, n):
    if n == 0 :
        return 0
    else:
        return alpha*q + beta*(q*q)/(2*n)

In [25]:
def create_strategy(num_periods):
    built_list = [random.randint(0,2) for _ in range(num_periods)]
    return built_list

In [26]:
def generate_population(num_periods, pop_size):
    return [create_strategy(num_periods) for _ in range(pop_size)]

In [27]:
def generate_children(population, num_children):
    children = []
    for i in range(math.ceil(num_children / 2)):
        parent_pair = random.sample(population, 2)
        children_pair = one_point_crossover(*parent_pair)
        for child in children_pair:
            children.append(mutation(child))
    return children[0:num_children]

In [28]:
def one_point_crossover(parent1, parent2):
    crossover_index = random.randint(0, len(parent1) - 1)
    children_pair = [(parent1[0:crossover_index] + parent2[crossover_index:]), \
         (parent2[0:crossover_index] + parent1[crossover_index:])]
    return children_pair

In [29]:
def mutation(strategy):
    N = 30
    prob_mutation = 1 / len(strategy)
    new_strategy = []
    for b in strategy:
        if random.random() <= prob_mutation:
            new_strategy.append(random.randint(0,2))
        else:
            new_strategy.append(b)
    return new_strategy

In [30]:
def better_strategy(strat1, strat2):
    if fitness(strat1) >= fitness(strat2):
        return strat1
    return strat2

In [31]:
#build cost not considered here
def profit(num_plant):
    A_0 = 420
    B = 1
    alpha = 10
    beta = 0.5
    fixed_costs = 1250
    build_costs = 2500
    c = 100     
    quantity = competitive_quantity(num_plant, A_0, alpha, beta, B, c)
    revenue = price(A_0, B, quantity)*quantity
    prod_cost = production_costs(quantity, alpha, beta, num_plant)
    profit = revenue - prod_cost - fixed_costs*num_plant
    return profit 

In [32]:
profit_table = []
p = 8
para = 1 / p
for n in range(111):
    profit_table.append([n*para, profit(n*para)])

In [33]:
def profit_objective(strategy):
    p = 8
    para = 1 / p
    list1 = [float(format( (para*m), '.2f')) for m in range(0, p)]
    list1.reverse()
    N = 30

    built_list = strategy[:]

    n_list = strategy[:]

    for i in range(N):
        if built_list[i] != 0:
            m = built_list[i]
            list2 = [k*m  for k in list1]
            for l in range(p):
                if i+l+1 <=N-1:
                    n_list[i+l+1] =n_list[i+l+1] + list2[l]
            
    strategy =  built_list[:]
    
    total_profit = []
    for per in range(len(strategy)):  
        total_profit.append(profit_table[round(n_list[per] / 0.1)][1])
    
    num_built = sum(strategy) 
    
    return sum(total_profit) - 2500*num_built


In [34]:
def profit_objective_list(strategy):
    p = 8
    para = 1 / p
    list1 = [float(format( (para*m), '.2f')) for m in range(0, p)]
    list1.reverse()
    N = 30

    built_list = strategy[:]

    n_list = strategy[:]

    for i in range(N):
        if built_list[i] != 0:
            m = built_list[i]
            list2 = [k*m  for k in list1]
            for l in range(p):
                if i+l+1 <=N-1:
                    n_list[i+l+1] =n_list[i+l+1] + list2[l]
            
    strategy =  built_list[:]     
       
    A_0 = 420
    B = 1
    alpha = 10
    beta = 0.5
    fixed_costs = 1250
    build_costs = 2500
    c = 100 #capacity    
    N = len(n_list)
    
    profit_period = []
    for i in range(N): 
        quantity = competitive_quantity(n_list[i], A_0, alpha, beta, B, c)
        revenue = price(A_0, B, quantity)*quantity
        prod_cost = production_costs(quantity, alpha, beta, n_list[i])
        build_cost = built_list[i]*2500
        profit = revenue - prod_cost - fixed_costs*n_list[i] - build_cost
        profit_period.append(profit)  
      
    return profit_period

In [35]:
def fitness(strategy):
    profit_sum = sum(profit_objective_list(strategy))    
    return profit_sum

In [36]:
def tournament_survival(population, fitnesses,  pop_size):
    new_population = []
    new_fitnesses = []
    for i in range(num_solutions):
        indices = [random.randrange(len(population)) for _ in range(2)]
        if fitnesses[indices[0]] > fitnesses[indices[1]]:
            new_population.append(population[indices[0]])
            new_fitnesses.append(fitnesses[indices[0]])
        else:
            new_population.append(population[indices[1]])
            new_fitnesses.append(fitnesses[indices[1]]) 
    return new_population

In [37]:
def survival_solutions(parents, parent_fitnesses, children, num_solutions):
    population = parents + children
    fitnesses = parent_fitnesses + [profit_objective(child) for child in children]
    new_population = []
    new_fitnesses = []
    for i in range(num_solutions):
        indices = [random.randrange(len(population)) for _ in range(2)]
        if fitnesses[indices[0]] > fitnesses[indices[1]]:
            new_population.append(population[indices[0]])
            new_fitnesses.append(fitnesses[indices[0]])
        else:
            new_population.append(population[indices[1]])
            new_fitnesses.append(fitnesses[indices[1]])            
    return new_population, new_fitnesses

In [38]:
def genetic_algorithm(num_solutions, num_children,num_periods, num_sims):
    new_population = generate_population(num_periods, num_solutions) #possible_solutions(num_solutions, num_periods) 
    new_fitnesses = [profit_objective(strategy) for strategy in new_population]
    for i in range(num_sims):
        parents = new_population
        parent_fitnesses = new_fitnesses
        children = generate_children(parents, num_children)
        new_population, new_fitnesses = survival_solutions(parents, parent_fitnesses, children, num_solutions)
    return new_population

In [39]:
def genetic_optimal(num_solutions, num_children, num_periods, num_sims):
    final_population = genetic_algorithm(num_solutions, num_children, num_periods, num_sims)
    best_solution = max(final_population, key = lambda final_population: profit_objective(final_population))
    return [profit_objective(best_solution), best_solution]

In [40]:
population_size = 1000 #1000, 500, 30, 300
num_children = 500
num_period = 30 
print(genetic_optimal(1000, 500, 30, 300))  

[965250.0, [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0]]
